# Experiment 1: LeNet5

In [1]:
# import modules
%matplotlib inline
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import glob
import imageio
from utils.data_u import set_targets, fill_targets, load_data_csv, train_val_test_split, load_images, get_model_params, restore_model_params, data_to_name_targets
import seaborn as sns
import time
from random import randint
%config IPCompleter.greedy=True

In [2]:
# Load CSV data
height, width, channels = 512, 512, 1
data_csv = load_data_csv("train.csv")
data_csv.head()

,Id,Target
0,00070df0-bbc3-11e8-b2bc-ac1f6b6435d0,16 0
1,000a6c98-bb9b-11e8-b2b9-ac1f6b6435d0,7 1 2 0
2,000a9596-bbc4-11e8-b2bc-ac1f6b6435d0,5
3,000c99ba-bba4-11e8-b2b9-ac1f6b6435d0,1
4,001838f8-bbca-11e8-b2bc-ac1f6b6435d0,18


In [3]:
# One-hot encode targets
data_set = set_targets(data_csv)
data_set = data_csv.apply(fill_targets, axis=1)
data_set.head()

,Id,Target,Nucleoplasm,Nuclear membrane,Nucleoli,Nucleoli fibrillar center,Nuclear speckles,Nuclear bodies,Endoplasmic reticulum,Golgi apparatus,...,Microtubule organizing center,Centrosome,Lipid droplets,Plasma membrane,Cell junctions,Mitochondria,Aggresome,Cytosol,Cytoplasmic bodies,Rods & rings
0,00070df0-bbc3-11e8-b2bc-ac1f6b6435d0,"[16, 0]",1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,000a6c98-bb9b-11e8-b2b9-ac1f6b6435d0,"[7, 1, 2, 0]",1,1,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,000a9596-bbc4-11e8-b2bc-ac1f6b6435d0,[5],0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,000c99ba-bba4-11e8-b2b9-ac1f6b6435d0,[1],0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,001838f8-bbca-11e8-b2bc-ac1f6b6435d0,[18],0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [4]:
file_names, targets = data_to_name_targets(data_set)
print(file_names[0:5])
print(targets[0:5])

['00070df0-bbc3-11e8-b2bc-ac1f6b6435d0_green.png'
 '000a6c98-bb9b-11e8-b2b9-ac1f6b6435d0_green.png'
 '000a9596-bbc4-11e8-b2bc-ac1f6b6435d0_green.png'
 '000c99ba-bba4-11e8-b2b9-ac1f6b6435d0_green.png'
 '001838f8-bbca-11e8-b2bc-ac1f6b6435d0_green.png']
[[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
 [1 1 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]]


In [5]:
# split into train and test sets
X_train_name, y_train, X_val_name, y_val, X_test_name, y_test = train_val_test_split(file_names, targets)
print("Train shape: " ,X_train_name.shape)
print("Val shape: " ,X_val_name.shape)
print("Test shape: " ,X_test_name.shape)

Train shape:  (21750,)
Val shape:  (4661,)
Test shape:  (4661,)


In [6]:
# Create random sample of train and validation to check the accuracy of the model
accuracy_check_set_size = 200
train_ran = randint(accuracy_check_set_size, X_train_name.shape[0])
check_train_set_accuracy_X = load_images(X_train_name, start=train_ran-accuracy_check_set_size, end=train_ran)
check_train_set_accuracy_y = y_train[train_ran-accuracy_check_set_size:train_ran]
val_ran = randint(accuracy_check_set_size, X_val_name.shape[0])
check_val_set_accuracy_X = load_images(X_val_name, start=val_ran-accuracy_check_set_size, end=val_ran)
check_val_set_accuracy_y = y_val[val_ran-accuracy_check_set_size:val_ran]
check_train_set_accuracy_X.shape

(200, 512, 512, 1)

### Modified LeNet5

In [7]:
scale = 0.001
tf.reset_default_graph()

with tf.name_scope("inputs"):
    X = tf.placeholder(tf.float32, shape=(None, height, width, 1), name="X")
    y = tf.placeholder(tf.float32, shape=[None, 28], name="y")
    training = tf.placeholder_with_default(False, shape=(), name='training')

with tf.name_scope("convNet"):
    conv1 = tf.layers.conv2d(inputs=X,
                             filters=6,
                             kernel_size=[5,5],
                             strides=[1,1],
                             padding="valid",
                             activation=tf.nn.relu,
                             kernel_initializer=tf.initializers.he_normal(),
                             kernel_regularizer=tf.contrib.layers.l1_regularizer(scale)
                        )
    conv1 = tf.layers.batch_normalization(conv1)
    pool1 = tf.layers.max_pooling2d(inputs=conv1,
                                    pool_size=[2,2],
                                    strides=[2,2],
                                    padding="valid"
                                   )
    
    conv2 = tf.layers.conv2d(pool1, 16, [5,5], strides=[1,1], padding="valid",
                             activation=tf.nn.relu,
                             kernel_initializer=tf.initializers.he_normal(),
                             kernel_regularizer=tf.contrib.layers.l1_regularizer(scale)
                            )
    conv2 = tf.layers.batch_normalization(conv2)
    pool2 = tf.layers.max_pooling2d(conv2, [2,2], strides=[2,2], padding="valid")
    
    pool2_flat = tf.layers.flatten(pool2)

with tf.name_scope("fc"):
    fc1 = tf.layers.dense(pool2_flat, units=120, activation=tf.nn.relu, 
                          kernel_initializer=tf.initializers.he_normal(),
                          kernel_regularizer=tf.contrib.layers.l1_regularizer(scale)
                         )
    dropfc1 = tf.layers.dropout(fc1, rate=0.95, training=training)
    fc2 = tf.layers.dense(dropfc1, units=84, activation=tf.nn.relu, 
                          kernel_initializer=tf.initializers.he_normal(),
                          kernel_regularizer=tf.contrib.layers.l1_regularizer(scale)
                         )
    dropfc2 = tf.layers.dropout(fc2, rate=0.95, training=training)
    
with tf.name_scope("output"):
    logits = tf.layers.dense(dropfc2, units=28)
    tf.summary.histogram('pre_activations', logits)
    result_tensor = tf.nn.sigmoid(logits)

with tf.name_scope("train"):
    entropy = tf.nn.sigmoid_cross_entropy_with_logits(logits=logits, labels=y)
    loss = tf.reduce_mean(entropy)
    tf.summary.scalar('cross_entropy', loss)
    optimizer = tf.train.AdamOptimizer(learning_rate=0.1)
    training_op = optimizer.minimize(loss)
    
with tf.name_scope("eval"):
    correct = tf.equal(tf.round(result_tensor), y)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    tf.summary.scalar('accuracy', accuracy)
    
with tf.name_scope("init_and_save"):
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()


In [8]:
init = tf.global_variables_initializer()

n_epochs = 50
image_subset_size = 2000
batch_size = 100

best_loss_val = np.infty
check_interval = 500
checks_since_last_progress = 0
max_checks_without_progress = 20
best_model_params = None 

start_time = time.time()

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        batch_count = 0
        batch_accuracy = 0;
        for image_load_offset in range(0,  len(X_train_name), image_subset_size):
            total_offset = image_load_offset+image_subset_size
            loading_image_time = time.time()
            X_train_subset = load_images(X_train_name, start=image_load_offset, end=total_offset)
            y_train_subset = y_train[image_load_offset:total_offset]
            # Split subset into bacth
            for bacth_offset in range(0, X_train_subset.shape[0], batch_size):
                batch_x = X_train_subset[bacth_offset:bacth_offset+batch_size]
                batch_y = y_train_subset[bacth_offset:bacth_offset+batch_size]
                # Run Training OP
                run_options = tf.RunOptions(report_tensor_allocations_upon_oom = True)
                sess.run(training_op, feed_dict={X: batch_x, y: batch_y, training: True}, options=run_options)
                # Check for progress
                if bacth_offset % check_interval == 0:
                    loss_val = loss.eval(feed_dict={X:batch_x,y:batch_y})
                    if loss_val < best_loss_val:
                        best_loss_val = loss_val
                        checks_since_last_progress = 0
                        best_model_params = get_model_params()
                    else:
                        checks_since_last_progress += 1
            X_train_subset = np.delete(X_train_subset, np.s_[:])
            X_train_subset = None
        # Check accuracy after epoch
        acc_train = accuracy.eval(feed_dict={X: check_train_set_accuracy_X, y: check_train_set_accuracy_y})
        acc_val = accuracy.eval(feed_dict={X: check_val_set_accuracy_X, y: check_val_set_accuracy_y})
        print(epoch, "Train accuracy:", acc_train, "Val accuracy:", acc_val)
    
    if best_model_params:
        restore_model_params(best_model_params)
    save_path = saver.save(sess, "./summary/model/leNet5.ckpt")

print("--- %s seconds total ---" % (time.time() - start_time))

Loading images for training subset
!DONE loading image subset, time= 4.691925048828125
Loading images for training subset
!DONE loading image subset, time= 4.6969380378723145
Loading images for training subset
!DONE loading image subset, time= 4.624675989151001
Loading images for training subset
!DONE loading image subset, time= 4.672062873840332
Loading images for training subset
!DONE loading image subset, time= 4.706315755844116
Loading images for training subset
!DONE loading image subset, time= 4.720272541046143
Loading images for training subset
!DONE loading image subset, time= 4.669011354446411
Loading images for training subset
!DONE loading image subset, time= 4.700692892074585
Loading images for training subset
!DONE loading image subset, time= 4.709497928619385
Loading images for training subset
!DONE loading image subset, time= 4.716193199157715
Loading images for training subset
!DONE loading image subset, time= 4.162497043609619
0 Train accuracy: 0.93892854 Val accuracy:

!DONE loading image subset, time= 4.074726581573486
Loading images for training subset
!DONE loading image subset, time= 4.568429708480835
Loading images for training subset
!DONE loading image subset, time= 4.554625988006592
Loading images for training subset
!DONE loading image subset, time= 4.347722768783569
Loading images for training subset
!DONE loading image subset, time= 4.326340913772583
Loading images for training subset
!DONE loading image subset, time= 4.460209608078003
Loading images for training subset
!DONE loading image subset, time= 4.505573987960815
Loading images for training subset
!DONE loading image subset, time= 4.748919725418091
Loading images for training subset
!DONE loading image subset, time= 4.720411777496338
Loading images for training subset
!DONE loading image subset, time= 4.182039022445679
8 Train accuracy: 0.93892854 Val accuracy: 0.9423214
Loading images for training subset
!DONE loading image subset, time= 4.561558246612549
Loading images for traini

Loading images for training subset
!DONE loading image subset, time= 4.154568433761597
Loading images for training subset
!DONE loading image subset, time= 4.277570009231567
Loading images for training subset
!DONE loading image subset, time= 4.154386043548584
Loading images for training subset
!DONE loading image subset, time= 4.190232753753662
Loading images for training subset
!DONE loading image subset, time= 4.286103010177612
Loading images for training subset
!DONE loading image subset, time= 4.184674978256226
Loading images for training subset
!DONE loading image subset, time= 4.303214073181152
Loading images for training subset
!DONE loading image subset, time= 3.813795804977417
16 Train accuracy: 0.93892854 Val accuracy: 0.9423214
Loading images for training subset
!DONE loading image subset, time= 4.0472259521484375
Loading images for training subset
!DONE loading image subset, time= 4.04057502746582
Loading images for training subset
!DONE loading image subset, time= 4.22374

!DONE loading image subset, time= 4.08530330657959
Loading images for training subset
!DONE loading image subset, time= 4.0615599155426025
Loading images for training subset
!DONE loading image subset, time= 4.073983430862427
Loading images for training subset
!DONE loading image subset, time= 4.104919910430908
Loading images for training subset
!DONE loading image subset, time= 4.175326347351074
Loading images for training subset
!DONE loading image subset, time= 4.26876974105835
Loading images for training subset
!DONE loading image subset, time= 3.8179359436035156
24 Train accuracy: 0.93892854 Val accuracy: 0.9423214
Loading images for training subset
!DONE loading image subset, time= 4.0438807010650635
Loading images for training subset
!DONE loading image subset, time= 4.046597003936768
Loading images for training subset
!DONE loading image subset, time= 4.08558464050293
Loading images for training subset
!DONE loading image subset, time= 4.027719259262085
Loading images for train

Loading images for training subset
!DONE loading image subset, time= 4.348477125167847
Loading images for training subset
!DONE loading image subset, time= 4.384902238845825
Loading images for training subset
!DONE loading image subset, time= 4.38878607749939
Loading images for training subset
!DONE loading image subset, time= 4.092409610748291
Loading images for training subset
!DONE loading image subset, time= 3.6305434703826904
32 Train accuracy: 0.93892854 Val accuracy: 0.9423214
Loading images for training subset
!DONE loading image subset, time= 4.0909340381622314
Loading images for training subset
!DONE loading image subset, time= 4.058859348297119
Loading images for training subset
!DONE loading image subset, time= 4.051882743835449
Loading images for training subset
!DONE loading image subset, time= 4.145336866378784
Loading images for training subset
!DONE loading image subset, time= 4.154834747314453
Loading images for training subset
!DONE loading image subset, time= 4.3617

!DONE loading image subset, time= 4.138559818267822
Loading images for training subset
!DONE loading image subset, time= 4.075792074203491
Loading images for training subset
!DONE loading image subset, time= 4.112748622894287
Loading images for training subset
!DONE loading image subset, time= 3.660861015319824
40 Train accuracy: 0.93892854 Val accuracy: 0.9423214
Loading images for training subset
!DONE loading image subset, time= 4.042657375335693
Loading images for training subset
!DONE loading image subset, time= 4.04636549949646
Loading images for training subset
!DONE loading image subset, time= 4.047739267349243
Loading images for training subset
!DONE loading image subset, time= 4.217272520065308
Loading images for training subset
!DONE loading image subset, time= 4.161473989486694
Loading images for training subset
!DONE loading image subset, time= 4.162582874298096
Loading images for training subset
!DONE loading image subset, time= 4.119992733001709
Loading images for traini

Loading images for training subset
!DONE loading image subset, time= 4.860868453979492
Loading images for training subset
!DONE loading image subset, time= 4.358642101287842
48 Train accuracy: 0.93892854 Val accuracy: 0.9423214
Loading images for training subset
!DONE loading image subset, time= 4.818758249282837
Loading images for training subset
!DONE loading image subset, time= 4.88461709022522
Loading images for training subset
!DONE loading image subset, time= 4.9029624462127686
Loading images for training subset
!DONE loading image subset, time= 4.795358896255493
Loading images for training subset
!DONE loading image subset, time= 4.788697957992554
Loading images for training subset
!DONE loading image subset, time= 4.778579235076904
Loading images for training subset
!DONE loading image subset, time= 4.675587892532349
Loading images for training subset
!DONE loading image subset, time= 4.33765721321106
Loading images for training subset
!DONE loading image subset, time= 4.604743

In [10]:
test_batch_size = 100
batch_count = 0
batch_accuracy = 0
image_subset_size = 2000
    
with tf.Session() as sess:
    saver.restore(sess, "./summary/model/leNet5.ckpt")    
    for image_load_offset in range(0,  len(X_train_name), image_subset_size):
        total_offset = image_load_offset+image_subset_size
        X_train_subset = load_images(X_train_name, start=image_load_offset, end=total_offset)
        y_train_subset = y_train[image_load_offset:total_offset]
        # Split subset into bacth
        for bacth_offset in range(0, X_train_subset.shape[0], batch_size):
            batch_x = X_train_subset[bacth_offset:bacth_offset+batch_size]
            batch_y = y_train_subset[bacth_offset:bacth_offset+batch_size]
            batch_count += 1
            batch_accuracy += accuracy.eval(feed_dict={X: batch_x, y: batch_y})
        X_train_subset = np.delete(X_train_subset, np.s_[:])
        X_train_subset = None
    print("Training set accuracy: ", batch_accuracy/batch_count)
    
    # Validation
    for image_load_offset in range(0,  len(X_val_name), image_subset_size):
        total_offset = image_load_offset+image_subset_size
        X_val_subset = load_images(X_val_name, start=image_load_offset, end=total_offset)
        y_val_subset = y_val[image_load_offset:total_offset]
        # Split subset into bacth
        for bacth_offset in range(0, X_val_subset.shape[0], batch_size):
            batch_x = X_val_subset[bacth_offset:bacth_offset+batch_size]
            batch_y = y_val_subset[bacth_offset:bacth_offset+batch_size]
            batch_count += 1
            batch_accuracy += accuracy.eval(feed_dict={X: batch_x, y: batch_y})
        X_val_subset = np.delete(X_val_subset, np.s_[:])
        X_val_subset = None
    print("Validation set accuracy: ", batch_accuracy/batch_count)
    
    # Test
    for image_load_offset in range(0,  len(X_test_name), image_subset_size):
        total_offset = image_load_offset+image_subset_size
        X_test_subset = load_images(X_test_name, start=image_load_offset, end=total_offset)
        y_test_subset = y_test[image_load_offset:total_offset]
        # Split subset into bacth
        for bacth_offset in range(0, X_test_subset.shape[0], batch_size):
            batch_x = X_test_subset[bacth_offset:bacth_offset+batch_size]
            batch_y = y_test_subset[bacth_offset:bacth_offset+batch_size]
            batch_count += 1
            batch_accuracy += accuracy.eval(feed_dict={X: batch_x, y: batch_y})
        X_test_subset = np.delete(X_test_subset, np.s_[:])
        X_test_subset = None
    print("Test set accuracy: ", batch_accuracy/batch_count)

INFO:tensorflow:Restoring parameters from ./summary/model/leNet5.ckpt
Training set accuracy:  0.9416399073163304
Validation set accuracy:  0.9416719976461159
Test set accuracy:  0.9416369604758728


### Modified AlexNet

In [7]:
# Model
scale = 0.001
tf.reset_default_graph()

with tf.name_scope("inputs"):
    X = tf.placeholder(tf.float32, shape=(None, height, width, 1), name="X")
    y = tf.placeholder(tf.float32, shape=[None, 28], name="y")
    training = tf.placeholder_with_default(False, shape=(), name='training')
    
with tf.name_scope("convNet"):
    c1 = tf.layers.conv2d(X,96,[11,11],[4,4],"same",
                          activation=tf.nn.relu
                        )
    c1 = tf.layers.batch_normalization(c1)
    p1 = tf.layers.max_pooling2d(c1,[3,3],[2,2],"valid")
    
    c2 = tf.layers.conv2d(p1,256,[5,5],[1,1],"same",
                          activation=tf.nn.relu
                        )
    p2 = tf.layers.max_pooling2d(c2,[3,3],[2,2],"valid")
    c3 = tf.layers.conv2d(p2,384,[3,3],[1,1],"same",
                          activation=tf.nn.relu
                        )
    c4 = tf.layers.conv2d(c3,384,[3,3],[1,1],"same",
                          activation=tf.nn.relu
                        )
    c5= tf.layers.conv2d(c4,256,[3,3],[1,1],"same",
                          activation=tf.nn.relu
                        )
    c5_flat = tf.layers.flatten(c5)

with tf.name_scope("fullyConnected"):
    # reduced the number of units, so it can run
    fc1 = tf.layers.dense(c5_flat, units=512, activation=tf.nn.relu)
    dropfc1 = tf.layers.dropout(fc1, rate=0.80, training=training)
    fc2 = tf.layers.dense(fc1, units=256, activation=tf.nn.relu)
    dropfc2 = tf.layers.dropout(fc2, rate=0.90, training=training)
    
with tf.name_scope("output"):
    logits = tf.layers.dense(dropfc2, units=28)
    result_tensor = tf.nn.sigmoid(logits)

with tf.name_scope("train"):
    entropy = tf.nn.sigmoid_cross_entropy_with_logits(logits=logits, labels=y)
    loss = tf.reduce_mean(entropy)
    optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
    training_op = optimizer.minimize(loss)
    
with tf.name_scope("eval"):
    correct = tf.equal(tf.round(result_tensor), y)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    
with tf.name_scope("init_and_save"):
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()

In [9]:
init = tf.global_variables_initializer()

n_epochs = 50
image_subset_size = 1000
batch_size = 100

best_loss_val = np.infty
check_interval = 500
checks_since_last_progress = 0
max_checks_without_progress = 20
best_model_params = None 

start_time = time.time()

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        batch_count = 0
        batch_accuracy = 0;
        for image_load_offset in range(0,  len(X_train_name), image_subset_size):
            total_offset = image_load_offset+image_subset_size
            X_train_subset = load_images(X_train_name, start=image_load_offset, end=total_offset)
            y_train_subset = y_train[image_load_offset:total_offset]
            # Split subset into bacth
            for bacth_offset in range(0, X_train_subset.shape[0], batch_size):
                batch_x = X_train_subset[bacth_offset:bacth_offset+batch_size]
                batch_y = y_train_subset[bacth_offset:bacth_offset+batch_size]
                # Run Training OP
                run_options = tf.RunOptions(report_tensor_allocations_upon_oom = True)
                sess.run(training_op, feed_dict={X: batch_x, y: batch_y, training: True}, options=run_options)
                # Check for progress
                if bacth_offset % check_interval == 0:
                    loss_val = loss.eval(feed_dict={X:batch_x,y:batch_y})
                    if loss_val < best_loss_val:
                        best_loss_val = loss_val
                        checks_since_last_progress = 0
                        best_model_params = get_model_params()
                    else:
                        checks_since_last_progress += 1
            X_train_subset = np.delete(X_train_subset, np.s_[:])
            X_train_subset = None
        # Check accuracy after epoch
        acc_train = accuracy.eval(feed_dict={X: check_train_set_accuracy_X, y: check_train_set_accuracy_y})
        acc_val = accuracy.eval(feed_dict={X: check_val_set_accuracy_X, y: check_val_set_accuracy_y})
        print(epoch, "Train accuracy:", acc_train, "Val accuracy:", acc_val)
    
    if best_model_params:
        restore_model_params(best_model_params)
    save_path = saver.save(sess, "./summary/model/alexnet.ckpt")

print("--- %s seconds total ---" % (time.time() - start_time))

0 Train accuracy: 0.94 Val accuracy: 0.9426786
1 Train accuracy: 0.94 Val accuracy: 0.9426786
2 Train accuracy: 0.94 Val accuracy: 0.9426786
3 Train accuracy: 0.94 Val accuracy: 0.9426786
4 Train accuracy: 0.94 Val accuracy: 0.9426786
5 Train accuracy: 0.94 Val accuracy: 0.9426786
6 Train accuracy: 0.94 Val accuracy: 0.9426786
7 Train accuracy: 0.9401786 Val accuracy: 0.9426786
8 Train accuracy: 0.9401786 Val accuracy: 0.9426786
9 Train accuracy: 0.94 Val accuracy: 0.9426786
10 Train accuracy: 0.9464286 Val accuracy: 0.94535714
11 Train accuracy: 0.945 Val accuracy: 0.94607145
12 Train accuracy: 0.9466072 Val accuracy: 0.94785714
13 Train accuracy: 0.94767857 Val accuracy: 0.94767857
14 Train accuracy: 0.94714284 Val accuracy: 0.9466072
15 Train accuracy: 0.94875 Val accuracy: 0.94696426
16 Train accuracy: 0.94946426 Val accuracy: 0.9473214
17 Train accuracy: 0.94946426 Val accuracy: 0.9473214
18 Train accuracy: 0.95107144 Val accuracy: 0.9482143
19 Train accuracy: 0.95089287 Val accur

In [9]:
test_batch_size = 100
batch_count = 0
batch_accuracy = 0
image_subset_size = 1000
batch_size = 100

with tf.Session() as sess:
    saver.restore(sess, "./summary/model/alexnet.ckpt")    
    for image_load_offset in range(0,  len(X_train_name), image_subset_size):
        total_offset = image_load_offset+image_subset_size
        X_train_subset = load_images(X_train_name, start=image_load_offset, end=total_offset)
        y_train_subset = y_train[image_load_offset:total_offset]
        # Split subset into bacth
        for bacth_offset in range(0, X_train_subset.shape[0], batch_size):
            batch_x = X_train_subset[bacth_offset:bacth_offset+batch_size]
            batch_y = y_train_subset[bacth_offset:bacth_offset+batch_size]
            batch_count += 1
            batch_accuracy += accuracy.eval(feed_dict={X: batch_x, y: batch_y})
        X_train_subset = np.delete(X_train_subset, np.s_[:])
        X_train_subset = None
    print("Training set accuracy: ", batch_accuracy/batch_count)
    
    # Validation
    for image_load_offset in range(0,  len(X_val_name), image_subset_size):
        total_offset = image_load_offset+image_subset_size
        X_val_subset = load_images(X_val_name, start=image_load_offset, end=total_offset)
        y_val_subset = y_val[image_load_offset:total_offset]
        # Split subset into bacth
        for bacth_offset in range(0, X_val_subset.shape[0], batch_size):
            batch_x = X_val_subset[bacth_offset:bacth_offset+batch_size]
            batch_y = y_val_subset[bacth_offset:bacth_offset+batch_size]
            batch_count += 1
            batch_accuracy += accuracy.eval(feed_dict={X: batch_x, y: batch_y})
        X_val_subset = np.delete(X_val_subset, np.s_[:])
        X_val_subset = None
    print("Validation set accuracy: ", batch_accuracy/batch_count)
    
    # Test
    for image_load_offset in range(0,  len(X_test_name), image_subset_size):
        total_offset = image_load_offset+image_subset_size
        X_test_subset = load_images(X_test_name, start=image_load_offset, end=total_offset)
        y_test_subset = y_test[image_load_offset:total_offset]
        # Split subset into bacth
        for bacth_offset in range(0, X_test_subset.shape[0], batch_size):
            batch_x = X_test_subset[bacth_offset:bacth_offset+batch_size]
            batch_y = y_test_subset[bacth_offset:bacth_offset+batch_size]
            batch_count += 1
            batch_accuracy += accuracy.eval(feed_dict={X: batch_x, y: batch_y})
        X_test_subset = np.delete(X_test_subset, np.s_[:])
        X_test_subset = None
    print("Test set accuracy: ", batch_accuracy/batch_count)

INFO:tensorflow:Restoring parameters from ./summary/model/alexnet.ckpt
Training set accuracy:  0.986977393200638
Validation set accuracy:  0.9792060678859926
Test set accuracy:  0.9738566445616575


In [43]:
import pandas as pd
import glob
import imageio

submission = pd.DataFrame(columns=['Id','Predicted'])
base_path = "data/test/"

with tf.Session() as sess:
    saver.restore(sess, "./summary/model/alexnet.ckpt")
    for file in list(glob.glob("data/test/*")):
        feed_data = np.zeros((1, height, width, 1))
        image = imageio.imread(file)
        feed_data[0, :, :, 0] = image
        result = tf.round(result_tensor.eval(feed_dict={X:feed_data})).eval(session=sess)
        result = np.ravel(result)
        real_result = np.ravel(np.argwhere(result == 1))
        real_result = ' '.join(map(str, real_result))
        temp = pd.DataFrame([[file_name[:-10], real_result]], columns=['Id','Predicted'])
        submission = submission.append(temp)
        

submission.head()

INFO:tensorflow:Restoring parameters from ./summary/model/alexnet.ckpt


KeyboardInterrupt: 

In [ ]:
submission['Predicted'].value_counts()